In [23]:
    import cv2
    import glob
    import argparse
    import math
    from numpy import genfromtxt
    import matplotlib.pyplot as plt
    import numpy as np
    import os.path
    from scipy import ndimage
    import os


    left = cv2.imread(r'C:\Users\K_kar\Downloads\imagesWithoutOcclusions_undistort\imagesWithoutOcclusions_undistort\left\1585434279_805531979_Left.png', cv2.IMREAD_UNCHANGED)
    right = cv2.imread(r'C:\Users\K_kar\Downloads\imagesWithoutOcclusions_undistort\imagesWithoutOcclusions_undistort\right\1585434279_805531979_Right.png', cv2.IMREAD_UNCHANGED)
    #left = (left/256).astype('uint8')
    #right = (right/256).astype('uint8')


    cameraMatrix1 = np.array([[1485.8503101355045, 0, 641.0072474534551], [0, 1486.8249802291273, 454.1981417235667], [0, 0, 1]])
    cameraMatrix2 = np.array([[1472.34425902698, 0, 656.7358738783742], [0, 1473.184475795988, 441.016803589085], [0, 0, 1]])
    distCoeffs1 = np.array([-0.09236217303671054, 0.15801009565677457, 0.0020679941868083445, -0.0023435708660260184, 0.04491629603683055])
    distCoeffs2 = np.array([-0.09949068652688753, 0.22953391558591676, 0.0016749995113326907, -0.0015940937703328348, -0.13603886268508916])
    rotationMatrix = np.array([[0.9999169807005986, 0.0026862926847088424, -0.012602203704541104],[-0.002633967055223802, 0.9999878496600472, 0.0041668633079119935],[0.012613243997904163, -0.004133323588458492, 0.9999119069757908]])
    transVector = np.array([29.96389633009774, 0.5883268401189343, -5.0370190999346365])
    essentialMatrix = np.array([[-0.005846632380824811, 5.0345261532342365, 0.6092635826971343], [-5.4145428656773165, 0.11031957194242471, -29.897779179091888], [-0.6672019134164675, 29.96195184048419, 0.1322696748639909]])
    fundMatrix = np.array([[4.567507458136527e-08, -3.930495370357416e-05, 0.010750771532659317], [4.227537878312907e-05, -8.607826196991683e-07, 0.3201405456504413], [-0.010999824926761303, -0.3182113833954986, 1]])


    flags = cv2.CALIB_ZERO_DISPARITY
    image_size = left.shape[::-1][1:3]
    print(image_size)

    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, image_size, rotationMatrix, transVector, flags = flags)

    leftmapX, leftmapY = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, image_size, cv2.CV_32FC1)
    rightmapX, rightmapY = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, image_size, cv2.CV_32FC1)

    left_remap = cv2.remap(left, leftmapX, leftmapY, cv2.INTER_LANCZOS4)
    right_remap = cv2.remap(right, leftmapX, rightmapY, cv2.INTER_LANCZOS4)


    # For some reason, the images get rotated upside down after remapping, and I have to invert them back
    #left_remap = ndimage.rotate(left_remap,180)
    #right_remap = ndimage.rotate(right_remap,180)
    # Find epilines corresponding to points in right image (second image) and
    # drawing its lines on left image
    #lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2 ,F)
    #lines1 = lines1.reshape(-1, 3)
    #left_remap = drawlines(left_remap, lines1, pts1, pts2)

    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    #lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    #lines2 = lines2.reshape(-1, 3)
    #right_remap = drawlines(right_remap, lines2, pts2, pts1)

    for line in range(0, int(right_remap.shape[0] / 20)):
        left_remap[line * 20, :] = 0
        right_remap[line * 20, :] = 0

    

(1280, 720)


In [31]:
# Arrays to store object points and image points from all the images.
objpoints_l = [] # 3d point in real world space
imgpoints_l = [] # 2d points in image plane.
objpoints_r = [] # 3d point in real world space
imgpoints_r = [] # 2d points in image plane.
left = glob.glob(r'C:\Users\K_kar\Downloads\imagesWithoutOcclusions_undistort\imagesWithoutOcclusions_undistort\left\1403709080387836928.png')
assert left
for fname in left:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret_l, mtx_l, dist_l, rvecs_l, tvecs_l = cv2.calibrateCamera(objpoints_l, imgpoints_l, gray.shape[::-1], None, None)
ret_r, mtx_r, dist_r, rvecs_r, tvecs_r = cv2.calibrateCamera(objpoints_r, imgpoints_r, gray.shape[::-1], None, None)

img_r = cv2.imread(r'C:\Users\K_kar\Downloads\imagesWithoutOcclusions_undistort\imagesWithoutOcclusions_undistort\right\1585434279_805531979_Right.png')
dst_r = cv2.undistort(img_r, mtx_r, dist_r, None, newcameramtx_r)

img_l = cv2.imread(r'C:\Users\K_kar\Downloads\imagesWithoutOcclusions_undistort\imagesWithoutOcclusions_undistort\left\1585434279_805531979_Left.png')
dst_l = cv2.undistort(img_l, mtx_l, dist_l, None, newcameramtx_l)

left_remap = dst_l
right_remap = dst_r

"""
Detect keypoints here
kp1, des1 = ...
kp2, des2 = ...
"""
sift = cv2.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(left_remap, None)
kp2, des2 = sift.detectAndCompute(right_remap, None)

kp_img = cv2.drawKeypoints(right_remap, kp2, right_remap, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize = (10,10))
plt.imshow(kp_img)

AssertionError: 

In [ ]:
cv2.namedWindow('output images', cv2.WINDOW_NORMAL)
cv2.imshow('output images', np.hstack([left_remap, right_remap]))
cv2.waitKey(0)
cv2.destroyAllWindows()